## ① Google Driveにアップロードしたテキスト（学習用データ）にアクセスできるようにする

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/'My Drive'
#%ls

/content/drive/My Drive


## ②globライブラリ
要インストール:ノート内で%pip install globとうって実行してください。

glo.glob関数を用いて、学習用に用いるテキストファイルをlist形式で得る。

In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
import glob
files = glob.glob("./西尾維新_化物語（下）.txt")
print(len(files))

1


## ③Mecabのインストールと形態素解析
以下のコマンド3行よりインストール。

Mecabを用いて文章を単語ごとに分割します、がその前に元の文章には不要な記号や記述、学習を妨げる特殊な文字（いわゆる外れ値）があるので、
まずreライブラリを用いて消去してます。

その後、Mecab.Tagger()ドライバを用いてフォルダにあるテキストファイル20個までについて形態素解析（単語へ分割）したのち、一つのリスト(text_all)へ格納。できればファイルの数だけ読み込みをしたかったがデータ量が莫大になってしまったため20個に断念。

In [5]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aptitude is already the newest version (0.8.13-3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-3)
git is already installed at the requested version (1:2.34.1-1ubuntu1.11)
make is already installed at the requested version (4.3-4.1build1)
curl is already installed at the requested version (7.81.0-1ubuntu1.17)
xz-utils is already installed at the requested version (5.2.5-2ubuntu1)
file is already installed at the requested version (1:5.41-3ubuntu0.1)
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the req

In [6]:
import re
def making_data(file):
  with open(file,mode='r',encoding='shift_jis', errors='ignore') as f:
    text_original = f.read()

  text = re.sub("《[^》]+》", "", text_original) # ルビの削除
  text = re.sub("［[^］]+］", "", text) # 読みの注意の削除
  text = re.sub(".*-", "", text, flags=re.DOTALL) #-より上の部分の削除
  text = re.sub("（[大正|明治.年]+.*", "", text, flags=re.DOTALL) #＃地付きより下の部分の削除
  text = re.sub("\n.\n", "", text, flags=re.DOTALL) #題名の削除
  text = re.sub("\n", "", text) #改行の削除
  text = re.sub("[0-9]+|[０-９]+", "", text, flags=re.DOTALL) #数字の削除
  text = re.sub("[Ａ-Ｚ]+|[A-Z]+|[a-z]+", "", text, flags=re.DOTALL) #全角英語の削除
  text = re.sub("[｜ 　―\,\，./「」\：\:（）＊×…※△『』〜＼／]", "", text) # 各種記号の削除
  text = re.sub("\[\［", "", text)
  text = re.sub("\]\］", "", text)





  return text

In [7]:
import MeCab
m = MeCab.Tagger('-Owakati')
text_all = []
# Iterate over the actual files in the list
for file in files:
    text = making_data(file)
    text = m.parse(text)
    text = text.split(' ')
    text_all.extend(text)

## ④使用されている単語の辞書の作成

得られた単語リストを集合に変換したのちソートをかけることで、辞書ができます。この辞書（chars）を用いて後にone-hot表現を実現します。

char_indicesはキーが文字、バリューがcharsのその文字のあるインデックス、indices_charはキーがインデックス、バリューがcharsのそのインデックスに対応した文字の辞書を定義しておきます。

In [8]:
chars = sorted(list(set(text_all)))
char_indices = {}
indices_char = {}

for i, char in enumerate(chars):
    char_indices[char] = i
    indices_char[i] = char

In [9]:
print(chars)

['\n', '!?', '、', '。', '々', 'あ', 'ああ', 'あいつ', 'あえて', 'あくまで', 'あけっぴろげ', 'あげ', 'あげる', 'あそこ', 'あたり', 'あっ', 'あっさり', 'あっち', 'あて', 'あてつけ', 'あと', 'あなた', 'あに', 'あの', 'あまり', 'あら', 'あらかた', 'あらそ', 'あららぎ', 'あり', 'ありがたい', 'ありがとう', 'ありのまま', 'ありゃ', 'ある', 'あるいは', 'ある程度', 'あれ', 'あろ', 'あん', 'あんた', 'あんな', 'あんなに', 'あんまり', 'あー', 'い', 'いい', 'いいえ', 'いい加減', 'いう', 'いえ', 'いえいえ', 'いか', 'いきなり', 'いく', 'いくら', 'いけ', 'いささか', 'いじめ', 'いそしむ', 'いたん', 'いちいち', 'いちゃつい', 'いっ', 'いっぱい', 'いつ', 'いつか', 'いつも', 'いひ', 'いや', 'いやあ', 'いやらしい', 'いら', 'いる', 'いれ', 'いろは', 'いわれ', 'いん', 'う', 'うう', 'ううん', 'うかがう', 'うっ', 'うるか', 'うるさい', 'うわ', 'うん', 'うーん', 'え', 'えい', 'ええ', 'えっ', 'えー', 'お', 'おい', 'おいしい', 'おお', 'おおぐま', 'おかしな', 'おく', 'おす', 'おっ', 'おとめ', 'おど', 'おどおど', 'おどろおどろ', 'おり', 'おろか', 'お前', 'お参り', 'お喋り', 'お姫様', 'お方', 'お母さん', 'お洒落', 'お父さん', 'お礼', 'お金', 'お陰', 'か', 'かい', 'かかっ', 'かかる', 'かけ', 'かさ', 'かざり', 'かしら', 'かじ', 'かなり', 'かに', 'かね', 'かぶる', 'かまけ', 'かも', 'から', 'からん', 'が', 'がたい', 'がっ', 'がっかり', 'がら', 'き', 'きかす', 'きっちり', 'きっと', 'きみ', 'きり', 'きれ', 'ぎ', '

## ⑤ニューロン数、バッチ数などの定義と学習用時系列データと正解データの作成
time_chars : 学習用データ : 時系列の数(n_rnn)だけ連続でテキストを入手、次の行には要素（単語）一つずらしたテキストが入る。これをもとの文章の長さ分だけやりたいが時系列の長さ分最後にはみ出てしまうので時系列の数分引いた数だけのループとなる。

next_chars : 正解データ : 学習用データの各行に対して次に並んでいる単語を格納。

これにより
学習用データには
\begin{bmatrix}
[text\_all[0] … text\_all[9]]\\
[text\_all[1] … text\_all[10]\\
[text\_all[len(text\_all)-n\_rnn] … text\_all[len(text\_all)-1]
\end{bmatrix}

正解データには
\begin{bmatrix}
[text\_all[10] \\
[text\_all[11]\\
[text\_all[len(text\_all)]]
\end{bmatrix}

In [10]:
n_rnn = 1180
n_in = 1
n_mid = 24
n_out = 1
epochs = 1
batch_size = 25

In [11]:
time_chars = []
next_chars = []
for i in range(0, len(text_all)-n_rnn):
    time_chars.append(text_all[i: i+n_rnn])
    next_chars.append(text_all[i + n_rnn])

In [12]:
len(time_chars)

13498

## ⑥データをOne-hot表現へ

正解用データ、学習用データそれぞれに単語辞書の長さ分の次元を追加する。
その入れ子をx、tとする。

一つ目のfor文にて学習用データの行の数だけループを回す。ここでchar_indicesを用いて正解データの辞書でのインデックスを取得し、tの該当するインデックスに1をセットする。これで各行の単語に該当するインデックスだけ1の配列が完成。

２つ目のfor分では学習用データの各行(i)に対して時系列の数だけループを回す。ここでも正解データと同様な操作を行い、時系列データそれぞれの単語に対応したインデックスが1となる配列の完成。


In [13]:
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=bool)
t = np.zeros((len(time_chars), len(chars)), dtype=bool)

for i,t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1

## ⑦RNNモデルの構築
RNNの構造についてSimpleRNN,LSTM,GRUの選択肢があったが計算量が少ないかつ精度が比較的良いGRUを用いることにした。
活性化関数や損失関数等はOne-hot表現に適したものを選んだ。

In [14]:
from keras.layers import GRU, Dense
from keras.models import Sequential

model_gru = Sequential()
model_gru.add(GRU(n_mid, input_shape=(n_rnn, len(chars))))
model_gru.add(Dense(len(chars), activation="softmax"))
model_gru.compile(loss='categorical_crossentropy', optimizer='adam')
print(model_gru.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 24)                156744    
                                                                 
 dense (Dense)               (None, 2151)              53775     
                                                                 
Total params: 210519 (822.34 KB)
Trainable params: 210519 (822.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## ⑧各エポックごとに学習度合いを確認する関数を作成

モデル学習時にLambdaCallbackインスタンスを渡すことでエポック前後での挙動を追加できる。今回はepoch終了時にon_epoch_end関数を実行することになってる。

またEarlystoppingインスタンスを渡すことにより学習が進まなくなった時に早期終了させることができる。今回はval_loss（検証用誤差）が変化しなくなってから5回epoch学習を行ったのち終了させる。

In [15]:
from keras.callbacks import LambdaCallback, EarlyStopping
import copy

def on_epoch_end(epoch, logs):
    print('エポック：', epoch)

    beta = 5
    index = np.random.choice(len(time_chars))
    prev_text = ''.join(text_all[index:n_rnn+index])  # リストを文字列に変換
    created_text = copy.copy(prev_text)

    model.save('bake.h5')

    print('シード：', created_text)

    for i in range(len(time_chars)):
        x_pred = np.zeros((1, n_rnn, len(chars)))

        for j, char in enumerate(prev_text):
            if j >= n_rnn:  # j が n_rnn を超えないように制限
                break
            if char in char_indices:
                x_pred[0, j, char_indices[char]] = 1
            else:
                print(f"Unknown character '{char}' encountered.")
                continue

        y = model.predict(x_pred)
        p_power = y[0] ** beta
        next_index = np.random.choice(len(chars), p=p_power/np.sum(p_power))
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char  # リストではなく文字列として操作

    print(created_text)
    print()



epoch_end_callback = LambdaCallback(on_epoch_end = on_epoch_end)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)


## ⑨モデルのロードと学習の実行
初めての学習であればmodel変数にはmodel_gruを入れます。
保存済みモデルを使う場合はコメント部分を使います。

In [16]:
print(f"type of time_chars: {type(time_chars)}")
print(f"length of time_chars: {len(time_chars)}")


type of time_chars: <class 'list'>
length of time_chars: 13498


In [17]:
!/opt/bin/nvidia-smi
!ps -aux|grep python
!kill -9 25937 25969 25986 26003 26033 26054 26337 26365 26404 26426 26444 26446
#from keras.models import load_model
# model = load_model('akutagawa.h5')
model = model_gru
history_gru = model.fit(x, t,
                        batch_size = batch_size,
                        epochs = epochs,
                        callbacks = [epoch_end_callback, early_stopping])

/bin/bash: line 1: /opt/bin/nvidia-smi: No such file or directory
root         552  0.0  0.0  66632 52052 ?        S    02:28   0:00 python3 /usr/local/bin/colab-file
root         577  0.5  0.0 2874020 91908 ?       Sl   02:28   0:13 /usr/bin/python3 /usr/local/bin/j
root        6527  0.0  0.0  18016  9544 ?        S    02:50   0:00 python3 /opt/google/drive/drive-f
root      220333  100 10.0 85773300 35360136 ?   SLsl 03:05   0:46 /usr/bin/python3 -m colab_kernel_
root      220412  0.3  0.0 539308 14484 ?        Sl   03:05   0:00 /usr/bin/python3 /usr/local/lib/p
root      221434  0.0  0.0   7372  3568 ?        S    03:06   0:00 /bin/bash -c ps -aux|grep python
root      221436  0.0  0.0   6480  2256 ?        S    03:06   0:00 grep python
/bin/bash: line 1: kill: (25937) - No such process
/bin/bash: line 1: kill: (25969) - No such process
/bin/bash: line 1: kill: (25986) - No such process
/bin/bash: line 1: kill: (26003) - No such process
/bin/bash: line 1: kill: (26033) - No such pro

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


ストリーミング出力は最後の 5000 行に切り捨てられました。
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 69ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 66ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 66ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 65ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 64ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 78ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 66ms/step
Unknown character '阿' encountered.
Unknown character '良' encountered.
1/1 [==============================] - 0s 68

に感情はないのだろうか。こっちは、ただの嫌がらせ返しだとわかっていても、そんなことを言われて、心臓がばくばくになっていると言うのに。わからないどうしてなんだどこでどう選択肢を間違えてしまって、僕はこんな茨の道を歩んでるんだいいじゃないの、茨の道。綺麗な薔薇の花が一杯咲いている道を優雅に歩いているイメージで、とてもゴージャスでビューティフルよいい解釈をしてんじゃねえ！薔薇の花言葉は愚かな男嘘をつけ！適当なことほざいてんじゃねえ！そういえばと、戦場ヶ原は言った。自分の都合でぽんぽん話題を変える奴だ。そういえば、ゴミいえ、阿良々木くん今お前、自分の彼氏をゴミと言いかけたか？何を言っているの、いわれのない言いがかりはやめて頂戴。そんなことより阿良々木くん、この前の実力テスト、どうだったの？あん？ほら、私が、私の家で、二人きりで、昼となく夜となく、散々面倒見てあげたじゃない何故わざわざそんな言い方をする。父親のいるところで、父親不在のときに家で二人きりになった話題など。テストは先週末には全て返ってきたというのに、阿良々木くんはその話題には触れたがらないから、これは酷い散々な結果だったのかしらと思って今日まで興味のない振りをしてきてあげたのだけれど、でも、今日、羽川さんに少し聞いたところ、そんな悪くはなかったらしいじゃない？羽川？あの子、さすがに口が堅いから、具体的には聞き出せなかったけれど、赤点だったとするなら教えてくれたはずだものそりゃあ嫌な訊き方をしたみたいだな。校門のところで、羽川が、戦場ヶ原についてなんだか不自然な話の振り方をしてきたと思ったが、なるほど、あのやり取りにはそういう伏線があったわけか。実力テストの結果については、昨日、あの本屋で、羽川には話したからなしかし、戦場ヶ原の物言いはともかくとして、あれだけ世話になっておいて一言の報告もなしというのは、確かに礼を欠くかもしれなかった。それこそ羽川に相談した、大学受験のことがあるから、なんとなく言えてなかったんだよな。何を言い惜しんでいるの。早く具体的な点数を教えなさい。勿体ぶっているようだと、身体中の関節を全部反対向きに折り曲げて、なんだか逆に格好いいみたいな体型にしてあげるわよその体型に格好いい要素はひとつもねえ！格好悪い？格好悪いなんてレベルの話じゃない！笑？笑えねえー！さあ、ブリッジでしか歩けない身体にされ

## ⑩モデルの保存
モデルの保存を行います。ただ、この操作はLambdaCallbackの関数にて指定しており毎epochごとに保存がなされますのでこの行で実行しなくてもいいです。

In [18]:
model.save('bake.h5')

In [19]:
#on_epoch_end(2, 3)